In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = ""
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
os.chdir(path)
import time
FOLDERNAME = "TED-OpenNMT-" + str(time.strftime("%Y%m%d-%H%M"))
!mkdir $FOLDERNAME

path = path + FOLDERNAME
os.chdir(path)
!pwd

# import os
# path = '/content/drive/Shared drives/chinh-share/nmt-v3/OpenNMT-TED-EM-bert-ratio-8-2-2-20210128-0637'
# os.chdir(path)
# !pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/TED-OpenNMT-20210409-1047


In [3]:
!nvidia-smi

Fri Apr  9 10:47:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 6.2MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 2.6MB 7.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=48229f6f8c88bec262d5259cd54d9123440d4d476231f59a24828bbdf0551d1c
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [20]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/en_vi_iwslt_bert.tar.gz
!mkdir data_bin
!tar -xvf 'en_vi_iwslt_bert.tar.gz'

--2021-04-09 10:51:21--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/en_vi_iwslt_bert.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22918476 (22M) [application/octet-stream]
Saving to: ‘en_vi_iwslt_bert.tar.gz’

en_vi_iwslt_bert.ta 100%[===================>]  21.86M  53.2MB/s    in 0.4s    

2021-04-09 10:51:21 (53.2 MB/s) - ‘en_vi_iwslt_bert.tar.gz’ saved [22918476/22918476]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [21]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-09 10:51:26,970 INFO] Extracting features...
[2021-04-09 10:51:26,974 INFO]  * number of source features: 0.
[2021-04-09 10:51:26,974 INFO]  * number of target features: 0.
[2021-04-09 10:51:26,974 INFO] Building `Fields` object...
[2021-04-09 10:51:26,974 INFO] Building & saving training data...
[2021-04-09 10:51:27,157 INFO] Building shard 0.
[2021-04-09 10:51:31,724 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-09 10:51:35,258 INFO]  * tgt vocab size: 18250.
[2021-04-09 10:51:35,318 INFO]  * src vocab size: 39660.
[2021-04-09 10:51:35,524 INFO] Building & saving validation data...
[2021-04-09 10:51:35,655 INFO] Building shard 0.
[2021-04-09 10:51:35,951 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [22]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-09 10:51:38,533 INFO]  * src vocab size = 39660
[2021-04-09 10:51:38,533 INFO]  * tgt vocab size = 18250
[2021-04-09 10:51:38,533 INFO] Building model...
[2021-04-09 10:51:46,438 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(39660, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [23]:
!ls -al model model/

model:
total 29897985
-rw------- 1 root root 1020517439 Apr  9 12:38 en-vi_step_10000.pt
-rw------- 1 root root 1020517439 Apr  9 11:02 en-vi_step_1000.pt
-rw------- 1 root root 1020517439 Apr  9 12:48 en-vi_step_11000.pt
-rw------- 1 root root 1020517439 Apr  9 12:59 en-vi_step_12000.pt
-rw------- 1 root root 1020517439 Apr  9 13:09 en-vi_step_13000.pt
-rw------- 1 root root 1020517439 Apr  9 13:20 en-vi_step_14000.pt
-rw------- 1 root root 1020517439 Apr  9 13:31 en-vi_step_15000.pt
-rw------- 1 root root 1020517439 Apr  9 13:41 en-vi_step_16000.pt
-rw------- 1 root root 1020517439 Apr  9 13:52 en-vi_step_17000.pt
-rw------- 1 root root 1020517439 Apr  9 14:03 en-vi_step_18000.pt
-rw------- 1 root root 1020517439 Apr  9 14:13 en-vi_step_19000.pt
-rw------- 1 root root 1020517439 Apr  9 14:24 en-vi_step_20000.pt
-rw------- 1 root root 1020517439 Apr  9 11:13 en-vi_step_2000.pt
-rw------- 1 root root 1020517439 Apr  9 14:34 en-vi_step_21000.pt
-rw------- 1 root root 1020517439 Apr  9 1

In [24]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-09 16:09:43,294 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [50], which does not match the required output shape [10, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [25]:
!tail vi_test

Cà vạt thì loè loẹt .
và lí do là bởi vì có 2 lí do , theo tôi nghĩ
Ông thích nói về thiên tài tâm linh của lứa tuổi .
Chúng tôi đều là người Triều Tiên , nhưng đã trở nên rất khác nhau do hậu quả của 67 năm bị chia cắt .
Đó là cách bạn xử lý một vấn đề khi bạn nhìn thấy chúng và đó không chỉ là việc than phiền về vấn đề đó .
Tham vọng của các bạn được thoã mãn , nó rất đẹp .
Không có thứ nào trong những điều trên thực sự hữu ích bởi vì bạn đang điều trị những triệu chứng chứ không phải nguyên nhân của các vấn đề cơ bản ở Phi Châu .
Nhưng hiện nay nhiều người sống đến 90 hay 100 tuổi , trừ khi họ bắt tay quá nhiều hay làm những điều đại loại thế .
Nhưng quý vị phải có những công cụ đúng .
Những điều này là một phần cuộc đời ông và là những gì ông còn nhớ


In [26]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17114 (delta 7), reused 8 (delta 4), pack-reused 17085
Receiving objects: 100% (17114/17114), 273.05 MiB | 19.04 MiB/s, done.
Resolving deltas: 100% (12323/12323), done.
Checking out files: 100% (228/228), done.


In [27]:
!ls -al

total 95226
drwx------  2 root root     4096 Apr  9 10:51 data_bin
-rw-------  1 root root   996149 Apr  9 10:08 en_test
-rw-------  1 root root  8024744 Apr  9 10:08 en_train
-rw-------  1 root root  8654114 Apr  9 10:22 en_train_EM_0.8
-rw-------  1 root root  8307178 Apr  9 10:22 en_train_EM_0.85
-rw-------  1 root root  8154398 Apr  9 10:22 en_train_EM_0.9
-rw-------  1 root root  8069415 Apr  9 10:22 en_train_EM_0.95
-rw-------  1 root root  3494604 Apr  9 10:22 en_train_EM_factor_0.8
-rw-------  1 root root  3360444 Apr  9 10:22 en_train_EM_factor_0.85
-rw-------  1 root root  3300104 Apr  9 10:22 en_train_EM_factor_0.9
-rw-------  1 root root  3266074 Apr  9 10:22 en_train_EM_factor_0.95
-rw-------  1 root root   857727 Apr  9 10:22 en_train_EM_score_0.8
-rw-------  1 root root   857727 Apr  9 10:22 en_train_EM_score_0.85
-rw-------  1 root root   857727 Apr  9 10:22 en_train_EM_score_0.9
-rw-------  1 root root   857727 Apr  9 10:22 en_train_EM_score_0.95
-rw-------  1 root roo

In [28]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 23.50, 59.6/33.5/19.7/11.8 (BP=0.900, ratio=0.905, hyp_len=220969, ref_len=244219)
